In [67]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.ensemble import RandomForestRegressor 

from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

In [4]:
def getData(ds):
    Product_Category_dummies = pd.get_dummies(ds['Product_Category'], drop_first=True)
    Loyalty_customer_dummies = pd.get_dummies(ds['Loyalty_customer'],drop_first=True)
    ds = ds.drop(["Loyalty_customer", "Product_Category", "Product_id","Customer_name"],axis=1);
    ds = pd.concat([ds,Loyalty_customer_dummies,Product_Category_dummies ], axis=1)
    ds['instock_year'] = ds['instock_date'].dt.month
    ds['instock_month'] = ds['instock_date'].dt.year
    ds = ds.drop('instock_date',axis=1)
    return ds

    

In [5]:
ds = pd.read_csv("./dataset/train.csv", parse_dates=['instock_date']);


In [6]:
ds = getData(ds)

In [7]:
ds.dtypes;

In [8]:
# ds['Cosmetics'].plot.line()

In [9]:
# sns.pairplot(ds)
# dsN = ds.drop('Selling_Price',axis=1)

In [10]:
# grr = pd.scatter_matrix(dsN, figsize=(15, 15), marker='o',
# hist_kwds={'bins': 20}, s=60, alpha=.8)

In [11]:
ds.isnull().values.any();
ds.isnull().sum();
# ds.dropna(subset=['charges_1','','','',]) 
ds = ds.dropna(); 
ds.isnull().values.any();
ds.isnull().sum();

In [12]:
y = ds['Selling_Price'];
X = ds.drop('Selling_Price',axis=1);
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:

 # create regressor object 
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0) 
  
# fit the regressor with x and y data 
regressor.fit(X_train, y_train)  

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [14]:
Y_pred = regressor.predict(X_test) 

In [15]:
Y_pred.shape

(1839,)

In [16]:
y_test.shape

(1839,)

In [18]:
errors = abs(Y_pred - y_test)

In [21]:
type(y_test)
actual = y_test

In [22]:
type(Y_pred)
predicsted =  pd.Series(Y_pred)
# out['error'] = errors

In [23]:
type(errors)

pandas.core.series.Series

In [52]:
act = actual.to_frame()
act.reset_index(drop=True, inplace=True)
pred = predicsted.to_frame()
pred.reset_index(drop=True, inplace=True)
err = errors.to_frame()
err.reset_index(drop=True, inplace=True)

In [53]:
pred.columns.values

array([0], dtype=int64)

In [54]:
out = act;
out['predictedSP'] = pred[0]
out['error'] = err['Selling_Price']

In [55]:
out.head()

,Selling_Price,predictedSP,error
0,3227.567211,3162.346848,65.220363
1,4651.518528,4512.701564,138.816964
2,6568.669184,6716.051228,147.382044
3,5614.654752,5596.202986,18.451766
4,1095.835545,1073.011037,22.824508


In [56]:
out.to_csv("out.csv",index=False)


In [65]:
from math import sqrt
mean_squared_error(y_test, Y_pred)
rms = sqrt(mean_squared_error(y_test, Y_pred))


In [66]:
rms

343.28118061761205

In [68]:
r2_score(y_test,Y_pred)

0.9862976169259313

In [69]:
r2_score(y_test,y_test)

1.0

In [112]:
testData1 = pd.read_csv("./dataset/test.csv", parse_dates=['instock_date']);
testData = getData(testData1);

In [97]:
# testData.isnull().sum();
testData['Stall_no'].fillna(12, inplace=True);
testData.fillna(0,inplace=True);
# type(testData['Stall_no'].value_counts().max())


In [99]:
# testData.isnull().sum()

In [104]:
xtestData = testData;

In [105]:
Y_pred_testData = regressor.predict(xtestData) 

In [108]:
predicstedTestData =  pd.Series(Y_pred_testData)
# out['error'] = errors

In [110]:
predicstedTestData = predicstedTestData.to_frame()
predicstedTestData.reset_index(drop=True, inplace=True)

In [113]:
predicstedTestData['Product_id'] = testData1['Product_id']

In [117]:
predicstedTestData.head
final_result = predicstedTestData.rename({0: 'Selling_Price'}, axis=1)  # new method


In [120]:
final_result.to_csv('FinalResult.csv')